# TripAdvisor restaurants info scraping
Takes a city as an argument and scrape the summary data of each restaurants of the city through the TA restaurants display pages.

Curate the raw dataset generated and aggregate them into one single dataset for all cities.

https://www.tripadvisor.fr/Restaurants-g274772-Krakow_Lesser_Poland_Province_Southern_Poland.html#EATERY_OVERVIEW_BOX

In [1]:
#! /usr/bin/env python3
# coding: utf-8

import requests
from bs4 import BeautifulSoup
import datetime
import pandas as pd
import numpy as np
import logging
import glob2

In [2]:
#Variables that will be used globally through the script
url0 = 'https://www.tripadvisor.com'
today = datetime.datetime.now()
today_date = str(today.year) + '/' + str(today.month) + '/' + str(today.day)

#Enable display of info messages
logging.basicConfig(level=logging.INFO)

## Scrape data from the summary list of restaurants

The TripAdvisor URL to scroll through the restaurants list is built as follow:
https://www.tripadvisor.com/RestaurantSearch-g1225481-oa15, where 
- g122548 is the id of the city
- oa30 is the variable to scroll through the pages, by incrementing by 30 to go to the next page.

Restaurants are naturally sorted by descending Ranking
The information is heterogenous: 
- all restaurants have name, id, URL
- not all have cuisine, rank, rate, reviews

### Scraper

In [22]:
def scraper(city):
    query = '/TypeAheadJson?action=API&startTime='+today_date+'&uiOrigin=GEOSCOPE&source=GEOSCOPE&interleaved=true&type=geo&neighborhood_geos=true&link_type=eat&details=true&max=12&injectNeighborhoods=true&query='+city
    url = url0 + query
    #Query the API ad get a JSON answer readable by Python as dictionnaries objects
    api_response = requests.get(url).json()
    geo = api_response['results'][0]['url']  #Get the URL from the results/1st element/Url key
    restaurants_url = url0 + geo
    logging.info("Scraping {} restaurants info".format(city))
    print(restaurants_url)

    #Prepare the scrolling requests using a URL such as
    #https://www.tripadvisor.com/RestaurantSearch-g1225481-oa15
    scroll_url0= 'https://www.tripadvisor.com/RestaurantSearch-'
    b = restaurants_url.find('-')
    e = restaurants_url.find('-', b+1)
    city_id = restaurants_url[b+1:e]
    
    #Initialize the lists of parameters to scrape and the dataframe containing all data
    inc_page=0
    resto_dict = {}
    dataset = pd.DataFrame(resto_dict)
    #columns=['Name', 'URL_TA', 'ID_TA', 'Rating', 'Ranking', 'Price Range', 'Cuisine Style', 'Number of Reviews', 'Reviews'])
    
    #Get the total number of pages
    r = requests.get(scroll_url0+city_id,
                     headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',},
                     cookies= {"SetCurrency":"EUR"})
    soup = BeautifulSoup(r.text, "lxml")
    page_tag = soup.find_all(class_="pageNumbers")[0] #tag that displays number of pages at bottom of webpage
    a_tags = page_tag.find_all('a')  #last item of the returned list is the last page button
    tot_pages=int(a_tags[-1].contents[0])  #integer from text content of the <a>
    logging.info("{} pages to explore".format(tot_pages))
    
    #Explore all the pages that display restaurants
    for page_index in range (1, tot_pages+1):
        
        #URL of the current webpage
        scroll_url = scroll_url0 + city_id + '-oa' + str(inc_page)
        print("Scraping page n°{}".format(page_index))
        print(scroll_url)

        #Scrape HTML content of the current webpage using the library BeautifulSoup
        r = requests.get(scroll_url,
                 headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',},
                cookies= {"SetCurrency":"EUR"})
        soup = BeautifulSoup(r.text, "lxml")


        #Restaurants list starts with tag <div id="EATERY_SEARCH_RESULTS">
        data_bloc = soup.find_all(attrs={"id": "EATERY_SEARCH_RESULTS"}) #contains the data bloc in a list object
        data_bloc = data_bloc[0]  #easier to manipulate

    #First restaurant of page has a particular class attribute
        if data_bloc.find_all(class_="listing rebrand listingIndex-1 first") != []:
            resto_soup = data_bloc.find_all(class_="listing rebrand listingIndex-1 first")[0]
        else:
            resto_soup = data_bloc.find_all(class_="listing rebrand first")[0]

        #Get the url, id and name of restaurants
        url_name_tag = resto_soup.find_all(class_="property_title")[0] #tag containing the data
        #Get restaurant URL
        resto_dict['URL_TA'] = url_name_tag.get('href')
        #Get the restaurant ID within its URL (-dxxxxxxx-Reviews)
        b = url_name_tag.get('href').find('-d')
        e= url_name_tag.get('href').find('-R')
        resto_dict['ID_TA'] = url_name_tag.get('href')[b+1:e]
        #Get names
        resto_dict['Name'] = url_name_tag.contents[0][1:-1]

        #Get the ranking of the restaurant
        if resto_soup.find_all(class_="popIndex rebrand popIndexDefault") != []:
            ranking_tag = resto_soup.find_all(class_="popIndex rebrand popIndexDefault")[0]
            resto_dict['Ranking'] = ranking_tag.contents[0][1:-1]
        else:
            resto_dict['Ranking'] = np.nan #put a NaN instead

        #Get the rating of the restaurant from <span> tags
        if resto_soup.find_all('span') != []:
            span_tags = resto_soup.find_all('span')
            for tag in span_tags:
                if tag.get('alt') is not None:
                    resto_dict['Rating'] = tag.get('alt')
        else:
            resto_dict['Rating'] = np.nan

        #Information from <div class="cuisines">  
        #!! some resaurants don't have pricerange nor cuisine styles, instead <div class="cuisine_margin">
        cuisines_tags = resto_soup.find_all(class_="cuisines") #1 element of the list is 1 restaurant
        if resto_soup.find_all(class_="cuisines") != []:
            for item in cuisines_tags:
                #Get price range from <span class="item price">
                if item.find(class_="item price") is not None:
                    price_range = item.find(class_="item price") #unique tag with price range
                    resto_dict['Price Range'] = price_range.contents[0]
                else:
                     resto_dict['Price Range'] = np.nan
                #Get cuisine styles from <span class="item cuisine"> tags (several/restaurant)
                if item.find_all(class_="item cuisine") != []:
                    cuisines = item.find_all(class_="item cuisine")  # list of <a> tags with the cuisine style as text
                    resto_dict['Cuisine Style'] = [tag.contents[0] for tag in cuisines]
                else:
                    resto_dict['Cuisine Style'] = np.nan

        #Get number of reviews
        if resto_soup.find_all(class_="reviewCount") != []:
            numb_tag = resto_soup.find_all(class_="reviewCount")[0]
            resto_dict['Number of Reviews'] = numb_tag.find('a').contents[0][1:-9]
        else:
            resto_dict['Number of Reviews'] = np.nan
            
        #Get 2 reviews (text+date) from <ul class="review_stubs review_snippets rebrand"> and <li> tags within
        ul_tags = resto_soup.find_all(class_="review_stubs review_snippets rebrand")
        if ul_tags != []:
            for reviews_set in ul_tags:
                rev_texts = reviews_set.find_all(dir="ltr")
                rev_dates = reviews_set.find_all(class_="date")
                resto_dict['Reviews'] = [[tag.find('a').contents[0] for tag in rev_texts], #text is in a <a> tag
                                          [tag.contents[0] for tag in rev_dates]]
        else:
            resto_dict['Reviews'] = np.nan
            
        #Append the dataset
        dataset = pd.concat([dataset, pd.DataFrame([resto_dict])])
            
    #For the rest of the list from 2 to 30:
        try:
            inc_rest = 0
            for i in range (2, 31):
                resto_dict = {}
                resto_bloc_id = "listing rebrand listingIndex-" + str(i)
                if data_bloc.find_all(class_=resto_bloc_id) != []:
                    resto_soup = data_bloc.find_all(class_=resto_bloc_id)[0] #Bloc for one restaurant

                    #Get the url, id and name of restaurants
                    url_name_tag = resto_soup.find_all(class_="property_title")[0] #tag containing the data
                    #Get restaurant URL
                    resto_dict['URL_TA'] = url_name_tag.get('href')
                    #Get the restaurant ID within its URL (-dxxxxxxx-Reviews)
                    b = url_name_tag.get('href').find('-d')
                    e= url_name_tag.get('href').find('-R')
                    resto_dict['ID_TA'] = url_name_tag.get('href')[b+1:e]
                    #Get names
                    resto_dict['Name'] = url_name_tag.contents[0][1:-1]

                    #Get the ranking of the restaurant
                    if resto_soup.find_all(class_="popIndex rebrand popIndexDefault") != []:
                        ranking_tag = resto_soup.find_all(class_="popIndex rebrand popIndexDefault")[0]
                        resto_dict['Ranking'] = ranking_tag.contents[0][1:-1]
                    else:
                        resto_dict['Ranking'] = np.nan

                    #Get the rating of the restaurant from <span> tags
                    span_tags = resto_soup.find_all('span')
                    if resto_soup.find_all('span') != []:
                        for tag in span_tags:
                            if tag.get('alt') is not None:
                                resto_dict['Rating'] = tag.get('alt')
                    else:
                        resto_dict['Rating'] = np.nan

                    #Information from <div class="cuisines">  
                    #!! some resaurants don't have pricerange nor cuisine styles, instead <div class="cuisine_margin">
                    cuisines_tags = resto_soup.find_all(class_="cuisines") #1 element of the list is 1 restaurant
                    if resto_soup.find_all(class_="cuisines") != []:
                        for item in cuisines_tags:
                            #Get price range from <span class="item price">
                            if item.find(class_="item price") is not None:
                                price_range = item.find(class_="item price") #unique tag with price range
                                resto_dict['Price Range'] = price_range.contents[0]
                            else:
                                resto_dict['Price Range'] = np.nan
                            #Get cuisine styles from <span class="item cuisine"> tags (several/restaurant)
                            if item.find_all(class_="item cuisine") != []:
                                cuisines = item.find_all(class_="item cuisine")  # list of <a> tags with the cuisine style as text
                                resto_dict['Cuisine Style'] = [tag.contents[0] for tag in cuisines]
                            else: 
                                resto_dict['Cuisine Style'] = np.nan

                    #Get number of reviews
                    if resto_soup.find_all(class_="reviewCount") != []:
                        numb_tag = resto_soup.find_all(class_="reviewCount")[0]
                        resto_dict['Number of Reviews'] = numb_tag.find('a').contents[0][1:-9]
                    else:
                        resto_dict['Number of Reviews'] = np.nan

                    #Get 2 reviews (text+date) from <ul class="review_stubs review_snippets rebrand"> and <li> tags within
                    ul_tags = resto_soup.find_all(class_="review_stubs review_snippets rebrand")
                    if resto_soup.find_all(class_="review_stubs review_snippets rebrand") != []:
                        for reviews_set in ul_tags:
                            rev_texts = reviews_set.find_all(dir="ltr")
                            rev_dates = reviews_set.find_all(class_="date")
                            #Able to pick up empty displayed review "" (St morris Argentijns, Amsterdam)
                            resto_dict['Reviews'] = [[tag.find('a').contents[0] if tag.find('a').contents != [] else np.nan for tag in rev_texts], #text is in a <a> tag
                                                  [tag.contents[0] for tag in rev_dates]]
                    else:
                        resto_dict['Reviews'] = np.nan

                    #Append the dataset
                    dataset = pd.concat([dataset, pd.DataFrame([resto_dict])])

                else: #tag of restaurant is instead "listing rebrand"
                    resto_soup = data_bloc.find_all(class_="listing rebrand")[inc_rest]

                    #Get the url, id and name of restaurants
                    url_name_tag = resto_soup.find_all(class_="property_title")[0] #tag containing the data
                    #Get restaurant URL
                    resto_dict['URL_TA'] = url_name_tag.get('href')
                    #Get the restaurant ID within its URL (-dxxxxxxx-Reviews)
                    b = url_name_tag.get('href').find('-d')
                    e= url_name_tag.get('href').find('-R')
                    resto_dict['ID_TA'] = url_name_tag.get('href')[b+1:e]
                    #Get names
                    resto_dict['Name'] = url_name_tag.contents[0][1:-1]

                    #Get the ranking of the restaurant
                    if resto_soup.find_all(class_="popIndex rebrand popIndexDefault") != []:
                        ranking_tag = resto_soup.find_all(class_="popIndex rebrand popIndexDefault")[0]
                        resto_dict['Ranking'] = ranking_tag.contents[0][1:-1]
                    else:
                        resto_dict['Ranking'] = np.nan

                    #Get the rating of the restaurant from <span> tags
                    span_tags = resto_soup.find_all('span')
                    if resto_soup.find_all('span') != []:
                        for tag in span_tags:
                            if tag.get('alt') is not None:
                                resto_dict['Rating'] = tag.get('alt')
                    else:
                        resto_dict['Rating'] = np.nan

                    #Information from <div class="cuisines">  
                    #!! some resaurants don't have pricerange nor cuisine styles, instead <div class="cuisine_margin">
                    cuisines_tags = resto_soup.find_all(class_="cuisines") #1 element of the list is 1 restaurant
                    if resto_soup.find_all(class_="cuisines") != []:
                        for item in cuisines_tags:
                            #Get price range from <span class="item price">
                            if item.find(class_="item price") is not None:
                                price_range = item.find(class_="item price") #unique tag with price range
                                resto_dict['Price Range'] = price_range.contents[0]
                            else:
                                resto_dict['Price Range'] = np.nan
                            #Get cuisine styles from <span class="item cuisine"> tags (several/restaurant)
                            if item.find_all(class_="item cuisine") != []:
                                cuisines = item.find_all(class_="item cuisine")  # list of <a> tags with the cuisine style as text
                                resto_dict['Cuisine Style'] = [tag.contents[0] for tag in cuisines]
                            else: 
                                resto_dict['Cuisine Style'] = np.nan

                    #Get number of reviews
                    if resto_soup.find_all(class_="reviewCount") != []:
                        numb_tag = resto_soup.find_all(class_="reviewCount")[0]
                        resto_dict['Number of Reviews'] = numb_tag.find('a').contents[0][1:-9]
                    else:
                        resto_dict['Number of Reviews'] = np.nan

                    #Get 2 reviews (text+date) from <ul class="review_stubs review_snippets rebrand"> and <li> tags within
                    ul_tags = resto_soup.find_all(class_="review_stubs review_snippets rebrand")
                    if resto_soup.find_all(class_="review_stubs review_snippets rebrand") != []:
                        for reviews_set in ul_tags:
                            rev_texts = reviews_set.find_all(dir="ltr")
                            rev_dates = reviews_set.find_all(class_="date")
                            #Able to pick up empty displayed review "" (St morris Argentijns, Amsterdam)
                            resto_dict['Reviews'] = [[tag.find('a').contents[0] if tag.find('a').contents != [] else np.nan for tag in rev_texts], #text
                                                  [tag.contents[0] for tag in rev_dates]]
                    else:
                        resto_dict['Reviews'] = np.nan

                    #Append the dataset
                    dataset = pd.concat([dataset, pd.DataFrame([resto_dict])])

                    inc_rest += 1

            #Increment to next page to display the next 30 restaurants
            inc_page += 30
    
        #End scrolling when no more restaurants when not able to find other restaurant bloc
        except IndexError:
            logging.info("Last restaurant reached")
            break
    
    #Save dataframe as csv file
    dataset.to_csv(city + '_TA_restaurants_raw.csv', sep=',', encoding="utf-8")
    print("File created in current directory: {}_TA_restaurants_raw.csv".format(city))

    return(dataset)

### Test on a middle-size city

In [ ]:
#Test
scraper('Krakow')

### Scrape all the euro capitals for restaurants data

In [ ]:
#Run the scraper to get data from all the euro capitals
euro_capitals = ['Paris', 'London', 'Budapest', 'Madrid', 'Lisbon', 'Berlin', 'Rome', 
            'Athens', 'Vienna', 'Warsaw', 'Ljubljana', 'Dublin',
                 'Bruxelles', 'Prague', 'Amsterdam', 'Luxembourg', 'Bratislava',
                'Copenhagen', 'Oslo', 'Helsinki', 'Stockholm']
for city in euro_capitals:
    scraper(city)

---

# Data Curation
## Exploration of raw datasets

In [25]:
#Explore the datasets obtained from the scraper
raw_csv_files = glob2.glob('*raw.csv')
print (raw_csv_files)
print("{} files in the directory".format(len(raw_csv_files)))
for file in raw_csv_files:
    print('\n' + file)
    dataset = pd.read_csv(file, sep=',',  encoding="utf-8")
    print(dataset.info())
    print(dataset.head())
    print(dataset.tail())

    #Count the unique values in Price range and rating columns
    print(dataset['Price Range'].value_counts(dropna=False))
    print(dataset['Rating'].value_counts(dropna=False))

['Amsterdam_TA_restaurants_raw.csv', 'Athens_TA_restaurants_raw.csv', 'Berlin_TA_restaurants_raw.csv', 'Bratislava_TA_restaurants_raw.csv', 'Bruxelles_TA_restaurants_raw.csv', 'Budapest_TA_restaurants_raw.csv', 'Copenhagen_TA_restaurants_raw.csv', 'Dublin_TA_restaurants_raw.csv', 'Helsinki_TA_restaurants_raw.csv', 'Koln_TA_restaurants_raw.csv', 'Krakow_TA_restaurants_raw.csv', 'Lisbon_TA_restaurants_raw.csv', 'Ljubljana_TA_restaurants_raw.csv', 'London_TA_restaurants_raw.csv', 'Luxembourg_TA_restaurants_raw.csv', 'Madrid_TA_restaurants_raw.csv', 'Oporto_TA_restaurants_raw.csv', 'Oslo_TA_restaurants_raw.csv', 'Paris_TA_restaurants_raw.csv', 'Prague_TA_restaurants_raw.csv', 'Rome_TA_restaurants_raw.csv', 'Stockholm_TA_restaurants_raw.csv', 'Toulouse_TA_restaurants_raw.csv', 'Vienna_TA_restaurants_raw.csv', 'Warsaw_TA_restaurants_raw.csv']
25 files in the directory

Amsterdam_TA_restaurants_raw.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3434 entries, 0 to 3433
Data columns (tot

      Unnamed: 0                                      Cuisine Style  \
7073           0  ['Mexican', 'Latin', 'Spanish', 'South America...   
7074           0                       ['German', 'Middle Eastern']   
7075           0                                       ['European']   
7076           0                                                NaN   
7077           0                                                NaN   

          ID_TA                      Name Number of Reviews Price Range  \
7073  d13350878  Chaparro Cocina Mexicana               NaN    $$ - $$$   
7074  d13353455                 Smoofee's               NaN    $$ - $$$   
7075  d13353635               L O F F E L               NaN    $$ - $$$   
7076  d13358302              Kandaka Food               NaN         NaN   
7077  d13358317                   Mayer's               NaN         NaN   

     Ranking Rating Reviews                                             URL_TA  
7073     NaN    NaN     NaN  /Restaurant_

Name: Price Range, dtype: int64
4 of 5 bubbles      1189
4.5 of 5 bubbles     719
3.5 of 5 bubbles     557
3 of 5 bubbles       237
5 of 5 bubbles       183
2.5 of 5 bubbles      41
NaN                   16
2 of 5 bubbles         3
Name: Rating, dtype: int64

Budapest_TA_restaurants_raw.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2606 entries, 0 to 2605
Data columns (total 10 columns):
Unnamed: 0           2606 non-null int64
Cuisine Style        2103 non-null object
ID_TA                2606 non-null object
Name                 2606 non-null object
Number of Reviews    2229 non-null object
Price Range          1740 non-null object
Ranking              2440 non-null object
Rating               2436 non-null object
Reviews              2437 non-null object
URL_TA               2606 non-null object
dtypes: int64(1), object(9)
memory usage: 203.7+ KB
None
   Unnamed: 0                                      Cuisine Style      ID_TA  \
0           0  ['Italian', 'Mediterranean', 'E

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2082 entries, 0 to 2081
Data columns (total 10 columns):
Unnamed: 0           2082 non-null int64
Cuisine Style        1735 non-null object
ID_TA                2082 non-null object
Name                 2082 non-null object
Number of Reviews    1868 non-null object
Price Range          1500 non-null object
Ranking              1954 non-null object
Rating               1955 non-null object
Reviews              1955 non-null object
URL_TA               2082 non-null object
dtypes: int64(1), object(9)
memory usage: 162.7+ KB
None
   Unnamed: 0                                      Cuisine Style     ID_TA  \
0           0  ['Irish', 'European', 'Contemporary', 'Vegetar...  d2239110   
1           0  ['Irish', 'European', 'Wine Bar', 'Central Eur...  d1587791   
2           0  ['Irish', 'European', 'Soups', 'Vegetarian Fri...  d3873866   
3           0  ['Irish', 'European', 'Contemporary', 'Vegetar...   d696187   
4           0  ['French', '

2027     NaN  /Restaurant_Review-g187371-d13344580-Reviews-C...  
NaN         1151
$$ - $$$     643
$            183
$$$$          51
Name: Price Range, dtype: int64
4 of 5 bubbles      636
4.5 of 5 bubbles    484
3.5 of 5 bubbles    326
NaN                 212
5 of 5 bubbles      173
3 of 5 bubbles      129
2.5 of 5 bubbles     41
2 of 5 bubbles       13
1 of 5 bubbles       10
1.5 of 5 bubbles      4
Name: Rating, dtype: int64

Krakow_TA_restaurants_raw.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1351 entries, 0 to 1350
Data columns (total 10 columns):
Unnamed: 0           1351 non-null int64
Cuisine Style        1049 non-null object
ID_TA                1351 non-null object
Name                 1351 non-null object
Number of Reviews    1150 non-null object
Price Range          897 non-null object
Ranking              1245 non-null object
Rating               1243 non-null object
Reviews              1243 non-null object
URL_TA               1351 non-null object
dtypes: int

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18212 entries, 0 to 18211
Data columns (total 10 columns):
Unnamed: 0           18212 non-null int64
Cuisine Style        14504 non-null object
ID_TA                18212 non-null object
Name                 18212 non-null object
Number of Reviews    15352 non-null object
Price Range          12163 non-null object
Ranking              16434 non-null object
Rating               16435 non-null object
Reviews              16436 non-null object
URL_TA               18212 non-null object
dtypes: int64(1), object(9)
memory usage: 1.4+ MB
None
   Unnamed: 0                                      Cuisine Style      ID_TA  \
0           0  ['Cafe', 'Middle Eastern', 'Persian', 'Vegetar...  d10517849   
1           0  ['Italian', 'Mediterranean', 'Vegetarian Frien...  d13149344   
2           0      ['Seafood', 'British', 'Gluten Free Options']  d12518072   
3           0  ['Mediterranean', 'European', 'Turkish', 'Midd...  d10444968   
4           

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9543 entries, 0 to 9542
Data columns (total 10 columns):
Unnamed: 0           9543 non-null int64
Cuisine Style        6402 non-null object
ID_TA                9543 non-null object
Name                 9543 non-null object
Number of Reviews    8232 non-null object
Price Range          4969 non-null object
Ranking              8803 non-null object
Rating               8812 non-null object
Reviews              8814 non-null object
URL_TA               9543 non-null object
dtypes: int64(1), object(9)
memory usage: 745.6+ KB
None
   Unnamed: 0                                      Cuisine Style      ID_TA  \
0           0  ['Mediterranean', 'European', 'Spanish', 'Cont...  d10428302   
1           0  ['International', 'European', 'Spanish', 'Cont...   d6884911   
2           0  ['International', 'Mediterranean', 'European',...   d2006562   
3           0  ['International', 'Mediterranean', 'European',...  d11896546   
4           0         

      Unnamed: 0                                      Cuisine Style  \
1208           0  ['Indian', 'Brazilian', 'South American', 'Cen...   
1209           0                                        ['Italian']   
1210           0                      ['Lebanese', 'Mediterranean']   
1211           0                                          ['Diner']   
1212           0  ['American', 'Fast Food', 'Asian', 'Grill', 'V...   

          ID_TA                          Name Number of Reviews Price Range  \
1208  d13219518         Oslo Smoothie Factory               NaN           $   
1209  d13290694  Piccolo Paradiso Pilestredet               NaN    $$ - $$$   
1210  d13298068               Vestlibua Pizza               NaN         NaN   
1211  d13329520         Northland, Gardermoen               NaN         NaN   
1212  d13355217           Stuffed Deli Bazaar               NaN         NaN   

     Ranking Rating Reviews                                             URL_TA  
1208     NaN    N

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2705 entries, 0 to 2704
Data columns (total 10 columns):
Unnamed: 0           2705 non-null int64
Cuisine Style        1815 non-null object
ID_TA                2705 non-null object
Name                 2705 non-null object
Number of Reviews    2266 non-null object
Price Range          1415 non-null object
Ranking              2461 non-null object
Rating               2463 non-null object
Reviews              2465 non-null object
URL_TA               2705 non-null object
dtypes: int64(1), object(9)
memory usage: 211.4+ KB
None
   Unnamed: 0                                      Cuisine Style     ID_TA  \
0           0  ['European', 'Swedish', 'Scandinavian', 'Veget...  d7345323   
1           0  ['European', 'Swedish', 'Scandinavian', 'Veget...  d5600577   
2           0  ['European', 'Swedish', 'Scandinavian', 'Inter...   d698819   
3           0  ['European', 'Spanish', 'Wine Bar', 'Vegetaria...  d2713535   
4           0  ['European',

$$ - $$$    1627
NaN         1597
$            392
$$$$         108
Name: Price Range, dtype: int64
4 of 5 bubbles      1203
4.5 of 5 bubbles    1031
3.5 of 5 bubbles     501
5 of 5 bubbles       390
NaN                  282
3 of 5 bubbles       207
2.5 of 5 bubbles      50
2 of 5 bubbles        40
1 of 5 bubbles        13
1.5 of 5 bubbles       6
-1 of 5 bubbles        1
Name: Rating, dtype: int64

Warsaw_TA_restaurants_raw.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2352 entries, 0 to 2351
Data columns (total 10 columns):
Unnamed: 0           2352 non-null int64
Cuisine Style        1692 non-null object
ID_TA                2352 non-null object
Name                 2352 non-null object
Number of Reviews    1941 non-null object
Price Range          1358 non-null object
Ranking              2154 non-null object
Rating               2146 non-null object
Reviews              2147 non-null object
URL_TA               2352 non-null object
dtypes: int64(1), object(9)
memory usage:

#### Problems to fix in the raws datasets:
- All data are object type 
    - the price range and rate are categorical variables, taking 3 and 9 values (NaN not included)
    - Rate, rank, number of reviews as numerical data
- No row Index, can used instead rank
- Unamed column to be deleted
- rearrange orger of the columns

## Dataset Curation & Aggregated Dataset Creation

Cures raw datasets to:
- have numerical useful values for number reviews, rate, rank
- have price range as a categorical type
- lists ready to be parsed
- rows in order and first empty row deleted
- add a column with city name for further concatenation

==> Creates a global dataset with concatenated curated dataset

In [26]:
#Curates all the raw.csv files in the current director and create a new curated & aggregated dataset    
raw_csv_files = glob2.glob('*raw.csv')
print(raw_csv_files)
curated_dataset = pd.DataFrame()

#Curates all the raw files from scraper
for file in raw_csv_files:
    city = file[:file.find('_')]
    print(city + ': Curating ' + file)
    dataset = pd.read_csv(file, sep=',',  encoding="utf-8")
    
    #broadcast the city name in the dataset
    dataset['City'] = city
    
    #Remove first empty column'Unamed: 0'
    dataset = dataset.drop('Unnamed: 0', axis=1)
    
    #Rating column into numerical data by slicing ' of 5 bubbles'
    dataset['Rating'] = dataset['Rating'].apply(lambda x: str(x)[:-13])
    dataset['Rating'] = pd.to_numeric(dataset['Rating'], errors='ignore')
    
    #Ranking column into numerical by slicing '#' and ' of xx restaurants in city'
    dataset['Ranking'] = dataset['Ranking'].apply(lambda x: str(x)[1:(str(x).find(' of'))].replace(',',''))
    dataset['Ranking'] = pd.to_numeric(dataset['Ranking'], errors='coerce')
    
    #Number of reviews column into numerical
    dataset['Number of Reviews'] =  dataset['Number of Reviews'].apply(lambda x: str(x).replace(',',''))
    dataset['Number of Reviews'] = pd.to_numeric(dataset['Number of Reviews'], errors='coerce')
    
    #Price range as categorical type:
    dataset['Price Range'] = dataset['Price Range'].astype('category')
    
    #Re-order columns
    dataset = dataset[['Name', 'City', 'Cuisine Style', 'Ranking', 'Rating', 
                       'Price Range', 'Number of Reviews', 'Reviews', 'URL_TA', 'ID_TA']]
    
    #Create a curated csv file
    dataset.to_csv(file.replace('raw', 'curated'),  sep=',', encoding="utf-8")
    print('Curated dataset created')

    #Append the curated dataset with the new curated dataset of the current city
    curated_dataset = pd.concat([curated_dataset, dataset])

#Create the aggregated curated csv file
curated_dataset.to_csv('TA_restaurants_curated.csv', sep=',', encoding="utf-8")
logging.info('Aggregated curated dataset created')
print(curated_dataset.info())

['Amsterdam_TA_restaurants_raw.csv', 'Athens_TA_restaurants_raw.csv', 'Berlin_TA_restaurants_raw.csv', 'Bratislava_TA_restaurants_raw.csv', 'Bruxelles_TA_restaurants_raw.csv', 'Budapest_TA_restaurants_raw.csv', 'Copenhagen_TA_restaurants_raw.csv', 'Dublin_TA_restaurants_raw.csv', 'Helsinki_TA_restaurants_raw.csv', 'Koln_TA_restaurants_raw.csv', 'Krakow_TA_restaurants_raw.csv', 'Lisbon_TA_restaurants_raw.csv', 'Ljubljana_TA_restaurants_raw.csv', 'London_TA_restaurants_raw.csv', 'Luxembourg_TA_restaurants_raw.csv', 'Madrid_TA_restaurants_raw.csv', 'Oporto_TA_restaurants_raw.csv', 'Oslo_TA_restaurants_raw.csv', 'Paris_TA_restaurants_raw.csv', 'Prague_TA_restaurants_raw.csv', 'Rome_TA_restaurants_raw.csv', 'Stockholm_TA_restaurants_raw.csv', 'Toulouse_TA_restaurants_raw.csv', 'Vienna_TA_restaurants_raw.csv', 'Warsaw_TA_restaurants_raw.csv']
Amsterdam: Curating Amsterdam_TA_restaurants_raw.csv
Curated dataset created
Athens: Curating Athens_TA_restaurants_raw.csv
Curated dataset created
Ber

INFO:root:Aggregated curated dataset created


<class 'pandas.core.frame.DataFrame'>
Int64Index: 99788 entries, 0 to 2351
Data columns (total 10 columns):
Name                 99788 non-null object
City                 99788 non-null object
Cuisine Style        75390 non-null object
Ranking              92270 non-null float64
Rating               92279 non-null float64
Price Range          62116 non-null category
Number of Reviews    86103 non-null float64
Reviews              92290 non-null object
URL_TA               99788 non-null object
ID_TA                99788 non-null object
dtypes: category(1), float64(3), object(6)
memory usage: 7.7+ MB
None
